In [ ]:
import random 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from scipy import stats

from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import precision_score, recall_score 
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix 
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, load_model 
from tensorflow.keras.layers import LSTM, Dense, Input 
from tensorflow.keras.regularizers import l1_12

from tqdm import tqdm 
import gc
import os

from joblib import load

import nbimporter 
import useful 
import create_model
import app_data

def manage_data(df_all, new_debut, column_name):
    """
    new_debut (int): l'indice du début des nouvelles données
    column_name (str): le nom de la colonne qui nous intéresse (bid ou ask)
    """

    # laisser que la nouvelle partie
    df_brut = df_all.iloc[new_debut:]
    df_brut.reset_index(drop=True, inplace=True)

    data_brut = df_brut[column_name]

    df_brut['index_column'] = df_brut.index

    return df_brut

def apply_model(df_all, new_debut, column_name, k_iqr, column, model, rf_classifier, lookback):
    df_brut = manage_data(df_all, new_debut, column_name)

    ############################### IQR
    # detecter les outliers avec IQR
    detected_outlier_indices = useful.detect_IQR(df_brut, k_iqr, column_name)

    # créer des labels et ecrire la liste des true outliers
    labels_first = df_brut['true_outliers']
    true_outlier_indices = df_brut[labels_first == 1].index.tolist()

    # la liste commune entre les outliers detectes avec IQR et les true outliers
    comm = [i for i in detected_outlier_indices.tolist() if i in true_outlier_indices]

    # calculer les metriques
    print("Metriques IQR")
    useful.afficher_mletriques(comm, true_outlier_indices, detected_outlier_indices.tolist(), column)

    ############################### update data after IQR
    # enlever les outliers deja detectes par le IQR et creer un nouveau dataframe df sans ces outliers aberrants
    df = df_brut.drop(detected_outlier_indices)
    df.reset_index(drop=True)

    # creer les nouveaux labels et les nouveaux indices des true outliers
    labels = df['true_outliers']
    outlier_indices = df[labels == 1].index.tolist()
    data = df[column_name]

    ############################### LSTM + RF
    # créer les séquences pour le LSTM
    X, y, y_labels = useful.create_sequences(data, labels, lookback)

    # reshape pour LSTM
    X_lstm = X.reshape(len(X), lookback, 1)

    # predictions avec le LSTM
    y_pred = model.predict(X, verbose=0).flatten()

    # calculer les résidus
    residuals = y - y_pred

    # preparer les données pour le classfier
    X_rf, y_rf = useful.prepare_classifier_data(X, residuals, y_pred, y, y_labels)

    # prediction sur le jeu de test
    y_rf_pred = rf_classifier.predict(X_rf)

    # les indices des outliers detectes par le classifier
    outlier_indices_rf = np.where(y_rf_pred == 1)[0]

    # mise à jour des indices des outliers detectes par le classifier
    anomaly_indices = np.arange(len(data)-lookback)
    detected_anomalies_global_indices = anomaly_indices[outlier_indices_rf] + lookback

    outliers_data_indices_df_brut_rf = df['index_column'][detected_anomalies_global_indices].tolist()
    outliers_data_indices_df_brut_rf.sort()

    detected_outliers_total = detected_outlier_indices.tolist() + outliers_data_indices_df_brut_rf
    detected_outliers_total.sort()

    comm_total = [i for i in detected_outliers_total if i in true_outlier_indices]

    print("Metriques TOTALES")
    useful.afficher_mletriques(comm_total, true_outlier_indices, detected_outliers_total, column)
    print(true_outlier_indices)
    print(detected_outliers_total)
    print(comm_total)

    return df




